# Deploying our model to an ACI (Azure Container Instance)


We now have a working model we will need to push it to a container in Azure so that it can be consumed

To build the correct environment for ACI, we need to provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model we trained before


## Connect to our Machine learning Workspace in Azure

You will need to modify the Config.Json to contain your Workspace name and other details

In [1]:
from azureml.core import Workspace, Experiment, Run
ws = Workspace.from_config()

Found the config file in: C:\Users\hannaj\Desktop\Test Jupyter Folder\Azure-ml Car Prediction\02. Model Deploy to ACI (Azure Container Instance)\config.json


## Create our scoring file

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:

The init() function, which typically loads the model into a global object. This function is run only once when the Docker container is started.

The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [2]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('car_prediction_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [3]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_pip_package("pynacl==1.2.1")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## Review contents of the file

In [4]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults
  - pynacl==1.2.1
- scikit-learn



# Create our config file

In [5]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "CarData",  "method" : "CarLearn"}, 
                                               description='Predict Car Prices with Linear Regression')

## Deploy into ACI

Estimated time to complete: about 7-8 minutes

Configure the image and deploy. The following code goes through these steps:

Build an image using:
        The scoring file (score.py)
        The environment file (myenv.yml)
        The model file
Register that image under the workspace.
Send the image to the ACI container.
Start up a container in ACI using the image.
Get the web service HTTP endpoint.

In [6]:
from azureml.core.image import ContainerImage
# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

In [7]:
from azureml.core.webservice import Webservice

service_name = 'car-predict-2'
service = Webservice.deploy(deployment_config = aciconfig,
                                image_config = image_config,
                                model_paths = ['../car_prediction_model.pkl'],
                                name = service_name,
                                workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

Registering model car_prediction_model.pkl
Creating image
Image creation operation finished for image car-predict-2:3, operation "Succeeded"
Creating service
Running.........................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


## Find the endpoint for our app

In [8]:
print(service.scoring_uri)

http://51.145.130.123:80/score


## Lets test our web service now to ensure it is working

In [9]:
import pickle
file_xtest = open('../X_test.pkl','rb')
X_test = pickle.load(file_xtest)

file_ytest = open('../y_test.pkl','rb')
y_test = pickle.load(file_ytest)

In [11]:
import json

test_samples = json.dumps({"data": X_test.tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

result = json.loads(service.run(input_data=test_samples))

print("Output of our Actual values:")
print(y_test[0:10].ravel())
print("Output of our Predicted values:")
print(result[0:10])

Output of our Actual values:
[ 7957 11199  5195  9279  6692  9549 10345 22470 12170 18399]
Output of our Predicted values:
[8988.387444508073, 15071.748488789664, 3229.7829312862086, 13328.97758458698, 7200.998410821521, 10728.014867986545, 9989.014477091761, 22582.372587438418, 13160.847276434906, 20131.792452553687]
